# Imports

In [1]:
import torch
import pennylane as qml
from pennylane import numpy as np
from loguru import logger
import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='IPython')

import sys
import os
import time

logger.info(f"Current directory: {os.getcwd()}")
sys.path.append("../")

from src.nn.encodings.pennylane_templates import angle_embedding, amplitude_embedding, QAOA_embedding
from src.nn.encodings.IQP_embedding import custom_iqp_embedding
from src.nn.encodings.NQE_embedding import NQE_embedding
from src.nn.encodings.ring_embedding import ring_embedding
from src.nn.encodings.waterfall_embedding import waterfall_embedding

from src.nn.ansatz.no_entanglement_circuit import no_entanglement_random_circuit
from src.nn.ansatz.full_entanglement_circuit import full_entanglement_circuit
from src.nn.ansatz.NQ_circuit import NQ_circuit
from src.nn.ansatz.ring_circuit import ring_circuit

from src.nn.measurements.default import default_measurement

from src.nn.models.hybrid.HQNN_Parallel import HQNN_Parallel
from src.utils.training import Trainer
from src.utils.dataset import load_dataset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.backends.cudnn.benchmark = True

2025-03-31 12:46:37.948 | INFO     | __main__:<module>:12 - Current directory: /Users/uribagi/Documents/GitHub/QML-Satellite-Image-Classification/notebooks


# Define embeddings

In [2]:
def build_embedding_configurations():
    """
    Dynamically create a list of embedding configurations based on parameter sweeps.
    """
    embedding_configurations = []

    # ----- Ring Embedding
    for n_repeats in [2]:
        embedding_configurations.append({
            "name": "ring",
            "func": ring_embedding,
            "func_params": {
                "n_repeats": n_repeats
            }
        })

    # ----- Waterfall Embedding
    embedding_configurations.append({
        "name": "waterfall",
        "func": waterfall_embedding,
        "func_params": {
            "weights": None
        }
    })

    # ----- Amplitude Embedding
    embedding_configurations.append({
        "name": "amplitude",
        "func": amplitude_embedding,
        "func_params": {
            "normalize": True,
            "pad_with": 0.0,
        }
    })

    # ----- Angle Embedding
    for rotation in ["X", "Y", "Z"]:
        embedding_configurations.append({
            "name": f"angle_{rotation}",
            "func": angle_embedding,
            "func_params": {
                "rotation": rotation
            }
        })

    # ----- IQP Embedding
    for repeats in [2]:
        embedding_configurations.append({
            "name": f"iqp_{repeats}",
            "func": custom_iqp_embedding,
            "func_params": {
                "n_repeats": repeats,
                "pattern": None
            }
        })

    # ----- NQE Embedding
    for repeats in [2]:
        embedding_configurations.append({
            "name": f"nqe_{repeats}",
            "func": NQE_embedding,
            "func_params": {
                "n_repeats": repeats
            }
        })

    # ----- QAOA Embedding
    for local_field in ["X", "Y", "Z"]:
        for n_layers in [2]:
            embedding_configurations.append({
                "name": f"qaoa_{local_field}_{n_layers}",
                "func": QAOA_embedding,
                "func_params": {
                    "weights": None,
                    "local_field": local_field,
                    "n_layers": n_layers
                }
            })



    return embedding_configurations

# Define circuits

In [3]:
def build_circuit_configurations():
    num_layers = 2
    num_qubits_per_circuit = 8
    weights_strongly_entangled = (2 * torch.rand(num_layers, num_qubits_per_circuit, 3) - 1) * np.pi
    weights_nq = (2 * torch.rand(3 * num_qubits_per_circuit, 2) - 1) * np.pi
    weights_no_ent = (2 * torch.rand(num_qubits_per_circuit, ) - 1) * np.pi


    configs = [{
        "name": f"no_entanglement",
        "func": no_entanglement_random_circuit,
        "func_params": {
            "num_layers": 1,
            "weights": weights_no_ent,
            "weight_shapes": {"weights": (num_qubits_per_circuit)},
        }
    }, {
        "name": f"full_entanglement",
        "func": full_entanglement_circuit,
        "func_params": {
            "num_layers": num_layers,
            "weights": weights_strongly_entangled,
            "weight_shapes": {"weights": (num_layers, num_qubits_per_circuit, 3)},
        }
    }, {
        "name": f"nq_circuit",
        "func": NQ_circuit,
        "func_params": {
            "num_layers": num_layers,
            "weights": weights_nq,
            "weight_shapes": {"weights": (num_layers, num_qubits_per_circuit, 2)},
        }
    }, {
        "name": f"ring_circuit",
        "func": ring_circuit,
        "func_params": {
            "weights": weights_strongly_entangled,
            "weight_shapes": {"weights": (num_layers, num_qubits_per_circuit, 3)},
        }
    }]

    # Full Entanglement
    # NQ circuit
    # Ring circuit
    return configs

# Define measurements

In [4]:
measurement_configurations = [
    {
        "name": "defaultZ",
        "func": default_measurement,
        "func_params": {"observable": qml.PauliZ}
    },
    {
        "name": "defaultX",
        "func": default_measurement,
        "func_params": {"observable": qml.PauliX}
    },
    {
        "name": "defaultY",
        "func": default_measurement,
        "func_params": {"observable": qml.PauliY}
    }
]

# Dataset

In [5]:
dataset_configurations = [
    {
        "dataset_name": "EuroSAT",
        "limit": 100,
        "image_size": 32,
        "test_size": 0.2,
        "output": "np",
        "allowed_classes": [
            'AnnualCrop', 'Forest', 'HerbaceousVegetation', 'Highway',
            'Industrial', 'Pasture', 'PermanentCrop', 'Residential', 'River', 'SeaLake'
        ]
    },
    {
        "dataset_name": "DeepSat4",
        "limit": 100,
        "image_size": 32,
        "test_size": 0.2,
        "output": "np",
        "allowed_classes": ['BarrenLand', 'Trees', 'Grassland', 'Other']
    },
    {
        "dataset_name": "DeepSat6",
        "limit": 100,
        "image_size": 32,
        "test_size": 0.2,
        "output": "np",
        "allowed_classes": ['BarrenLand', 'Trees', 'Grassland', 'Roads', 'Buildings' , 'WaterBodies']
    },
    {
        "dataset_name": "EuroSAT",
        "limit": 100,
        "image_size": 16,
        "test_size": 0.2,
        "output": "np",
        "allowed_classes": [
            'AnnualCrop', 'Forest', 'HerbaceousVegetation', 'Highway',
            'Industrial', 'Pasture', 'PermanentCrop', 'Residential', 'River', 'SeaLake'
        ]
    },
    {
        "dataset_name": "DeepSat4",
        "limit": 100,
        "image_size": 16,
        "test_size": 0.2,
        "output": "np",
        "allowed_classes": ['BarrenLand', 'Trees', 'Grassland', 'Other']
    },
    {
        "dataset_name": "DeepSat6",
        "limit": 100,
        "image_size": 16,
        "test_size": 0.2,
        "output": "np",
        "allowed_classes": ['BarrenLand', 'Trees', 'Grassland', 'Roads', 'Buildings' , 'WaterBodies']
    }
]

# Hyperparameters

In [6]:
hyperparameter_configurations = [
    {
        "epochs": 20,
        "learning_rate": 0.01,
        "early_stopping": True,
        "patience": 3,
        "use_schedulefree": True,
        "use_quantum": False,
        "plot": False,
        "log_mlflow": False
    },
    {
        "epochs": 20,
        "learning_rate": 0.01,
        "early_stopping": True,
        "patience": 3,
        "use_schedulefree": True,
        "use_quantum": True,
        "plot": False,
        "log_mlflow": True
    }

    # {
    #    "epochs": 20,
    #    "learning_rate": 0.001,
    #    "early_stopping": False,
    #    "patience": 5,
    #    "use_schedulefree": False,
    #    "use_quantum": False,
    #    "plot": False,
    #    "log_mlflow": False
    # },

]

# Helper function

In [7]:
def run_experiment(
    dataset_cfg,
    embedding_cfg,
    circuit_cfg,
    measurement_cfg,
    hparams
):
    """
    Prepare data, create model, trainer, and run training for one combination of config.
    """
    # Unpack dataset settings
    dataset_name = dataset_cfg["dataset_name"]
    limit = dataset_cfg["limit"]
    image_size = dataset_cfg["image_size"]
    test_size = dataset_cfg["test_size"]
    output = dataset_cfg["output"]
    allowed_classes = dataset_cfg["allowed_classes"]
    n_classes = len(allowed_classes)

    # Unpack hyperparameters
    epochs = hparams["epochs"]
    lr = hparams["learning_rate"]
    early_stopping = hparams["early_stopping"]
    patience = hparams["patience"]
    use_schedulefree = hparams["use_schedulefree"]
    use_quantum = hparams["use_quantum"]
    plot = hparams["plot"]
    log_mlflow = hparams["log_mlflow"]

    # The circuit dictionary also includes the chosen qkernel_shape
    if use_quantum:
        # Loguru info: Start of run
        logger.info(f"Starting run: dataset={dataset_name}, "
                f"embedding={embedding_cfg['name']}, "
                f"circuit={circuit_cfg['name']}, measurement={measurement_cfg['name']}, "
                f"epochs={epochs}, lr={lr}")

        run_name = (
            f"HQNN_Parallel_{dataset_name}_{image_size}x{image_size}_"
            f"emb={embedding_cfg['name']}_circuit={circuit_cfg['name']}_meas={measurement_cfg['name']}_"
            f"lr={lr}_ep={epochs}"
        )
    # Create a dictionary of all configurations for MLflow
        mlflow_params = {
            # Dataset parameters
            "dataset_name": dataset_name,
            "limit": limit,
            "image_size": image_size,
            "test_size": test_size,
            "allowed_classes": str(allowed_classes),  # Convert list to string

            # Embedding parameters
            "embedding_name": embedding_cfg['name'],

            # Circuit parameters
            "circuit_name": circuit_cfg['name'],

            # Measurement parameters
            "measurement_name": measurement_cfg['name'],

            # Any other relevant parameters you want to track
            "run_timestamp": time.strftime("%Y-%m-%d %H:%M:%S"),
        }

        embedding_params={
            "func": embedding_cfg["func"],
            "func_params": embedding_cfg["func_params"]
        }

        variational_params={
                "func": circuit_cfg["func"],
                "func_params": circuit_cfg["func_params"]  # includes 'weights' re-init
            }
        measurement_params={
            "func": measurement_cfg["func"],
            "func_params": measurement_cfg["func_params"]
        }

    else:
        logger.info(f"Starting Classic run: dataset={dataset_name}, "
                f"epochs={epochs}, lr={lr}")

        run_name = (
            f"HQNN_Parallel_{dataset_name}_{image_size}x{image_size}_"
            f"classic_"
            f"lr={lr}_ep={epochs}"
        )
        mlflow_params = {}
        embedding_params={}
        variational_params={}
        measurement_params={}

    mlflow_project_name = f"{dataset_name} {image_size}x{image_size}"

    # 1. Load Dataset
    train_loader, val_loader = load_dataset(
        dataset_name,
        output,
        limit,
        allowed_classes,
        image_size,
        test_size,
    )

    # 2. Create model
    model = HQNN_Parallel(
        embedding_params=embedding_params,
        variational_params=variational_params,
        measurement_params=measurement_params,
        n_classes=n_classes,
        use_quantum=use_quantum,
        dataset=dataset_name,
        input_size=image_size
    )

    # 3. Create Trainer
    trainer = Trainer(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        epochs=epochs,
        early_stopping=early_stopping,
        patience=patience,
        log=log_mlflow,
        mlflow_project=mlflow_project_name,
        mlflow_run_name=run_name,
        use_quantum=use_quantum,
        plot=plot,
        allowed_classes=allowed_classes,
        lr=lr,
        use_schedulefree=use_schedulefree,
        mlflow_params=mlflow_params,
    )

    logger.debug(f"Trainer created: early_stopping={early_stopping}, "
                 f"patience={patience}, log_mlflow={log_mlflow}")

    # 4. Train
    trainer.fit()

    logger.info(f"Finished run: {run_name}")

# Main loop

In [8]:
# 1. Build all embedding configs (with angle, iqp, nqe, qaoa sweeps, etc.)
dynamic_embedding_configurations = build_embedding_configurations()

# 2. Build circuit configs for qkernel_shape in [2,3,5]
circuit_configurations = build_circuit_configurations()

# 3. Nested loops
for hp_cfg in hyperparameter_configurations:
    for dataset_cfg in dataset_configurations:
        if not hp_cfg["use_quantum"]:
            # Run the experiment
            run_experiment(
                dataset_cfg=dataset_cfg,
                embedding_cfg={},
                circuit_cfg={},
                measurement_cfg={},
                hparams=hp_cfg
            )
            continue

        for emb_cfg in dynamic_embedding_configurations:
            for cir_cfg in circuit_configurations:
                for meas_cfg in measurement_configurations:
                    '''
                    # setting device on GPU if available, else CPU
                    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
                    print('Using device:', device)
                    print()

                    #Additional Info when using cuda
                    if device.type == 'cuda':
                        print(torch.cuda.get_device_name(0))
                        print('Memory Usage:')
                        print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**2,1), 'MB')
                        print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**2,1), 'MB')
                    '''
                    # Run the experiment
                    run_experiment(
                        dataset_cfg=dataset_cfg,
                        embedding_cfg=emb_cfg,
                        circuit_cfg=cir_cfg,
                        measurement_cfg=meas_cfg,
                        hparams=hp_cfg
                    )

2025-03-31 12:46:39.609 | INFO     | __main__:run_experiment:33 - Starting run: dataset=EuroSAT, embedding=ring, circuit=no_entanglement, measurement=defaultZ, epochs=1, lr=0.01
2025-03-31 12:47:27.585 | DEBUG    | __main__:run_experiment:135 - Trainer created: early_stopping=True, patience=3, log_mlflow=False
2025-03-31 12:48:55.652 | DEBUG    | src.utils.training:fit:226 - Epoch [1/1]: Train Loss = 4.1048, Train Acc = 15.38%, Val Loss = 2.0554, Val Acc = 22.50%
2025-03-31 12:48:55.685 | INFO     | __main__:run_experiment:141 - Finished run: HQNN_Parallel_EuroSAT_32x32_emb=ring_circuit=no_entanglement_meas=defaultZ_lr=0.01_ep=1
2025-03-31 12:48:55.690 | INFO     | __main__:run_experiment:33 - Starting run: dataset=EuroSAT, embedding=ring, circuit=no_entanglement, measurement=defaultX, epochs=1, lr=0.01
2025-03-31 12:49:43.828 | DEBUG    | __main__:run_experiment:135 - Trainer created: early_stopping=True, patience=3, log_mlflow=False
2025-03-31 12:51:27.373 | DEBUG    | src.utils.trai

IndexError: index 2 is out of bounds for dimension 0 with size 2